#1.IMPORT THE TENSOR FLOW

In [ ]:
pip install tensorflow

#2.Import nessary libraries


In [ ]:
#import nessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model


# 2.Dataset downloading

In [ ]:
#dataset downloading
import os
import glob
import requests
import zipfile
import io

# Step 1: Download the dataset ZIP from GitHub
url = "https://github.com/xmartlabs/malaria-detector/archive/refs/heads/master.zip"
print("Downloading dataset...")
response = requests.get(url)
if response.status_code == 200:
    print("Download successful, extracting files...")
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(".")
    print("Extraction completed.")
else:
    print("Failed to download dataset.")
    exit(1)

# Step 2: Define dataset directories after extraction
ROOT_DATA_DIR = './malaria-detector-master/cell_images'
INFECTED_DIR = os.path.join(ROOT_DATA_DIR, 'Parasitized')
UNINFECTED_DIR = os.path.join(ROOT_DATA_DIR, 'Uninfected')

# Step 3: Get list of image files
infected_files = glob.glob(os.path.join(INFECTED_DIR, '*.png'))
uninfected_files = glob.glob(os.path.join(UNINFECTED_DIR, '*.png'))

# Step 4: Print the counts
print(f'Amount of parasitized images: {len(infected_files)}')
print(f'Amount of uninfected images: {len(uninfected_files)}')
print(f'Total Images: {len(infected_files) + len(uninfected_files)}')


Download successful, extracting files...
Extraction completed.
Amount of parasitized images: 13779
Amount of uninfected images: 13779
Total Images: 27558


#3.To mount the google drive for dataset

In [ ]:
#to mount the google drive for dataset
from google.colab import drive
drive.mount('/content/drive')

# Then copy dataset folder to Drive, e.g.:
!cp -r ./malaria-detector-master /content/drive/MyDrive/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
^C


#4.Building the model

In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Load InceptionV3 base model (without top layers)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Make base model trainable
base_model.trainable = True

# Build the full model
model = Sequential([
    base_model,
    Flatten(),                       # Output: (None, 8192)
    Dense(1024, activation='relu'),  # Output: (None, 1024)
    Dropout(0.5),
    Dense(1, activation='sigmoid')   # Binary classification
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 2, 2, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     8,389,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,193,441 (115.18 MB)

 Trainable params: 30,159,009 (115.05 MB)

 Non-trainable params: 34,432 (134.50 KB)

#5. Data preparation and augmentation

In [ ]:
# Data preparation and augmentation
# Data Augmentation & Normalization Setup
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

#Training Data Preparation
train_generator = train_datagen.flow_from_directory(
    './malaria-detector-master/cell_images',
    target_size=(128, 128),  # match model input
    batch_size=32,
    class_mode='binary',
    subset='training'
)

#Validation Data Preparation
val_generator = train_datagen.flow_from_directory(
    './malaria-detector-master/cell_images',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 22048 images belonging to 2 classes.
Found 5510 images belonging to 2 classes.


#6.Fitting the model

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,
    verbose=1
)


Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


656/689 ━━━━━━━━━━━━━━━━━━━━ 2:57 5s/step - accuracy: 0.8884 - loss: 0.2736

# 7.Plot accuracy

In [ ]:
# Plot accuracy
plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy', marker='o')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot loss
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train Loss', marker='o')
plt.plot(history.history['val_loss'], label='Validation Loss', marker='o')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

#8.Single image and multiple image processing

In [ ]:
#single image and multiple image processing
import os
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load your trained model
model = load_model('/content/drive/MyDrive/InceptionV3.h5')

def preprocess_and_predict(img_path):
    try:
        img = image.load_img(img_path, target_size=(128, 128))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = img_array / 255.0
        prediction = model.predict(img_array)[0][0]
        label = 'Infected' if prediction >= 0.5 else 'Uninfected'
        return float(prediction), label
    except Exception as e:
        print(f" Error processing {img_path}: {e}")
        return None, None

choice = input("Choose prediction type (enter number):\n1. Single Image\n2. Folder of Images\nYour choice: ")

if choice == '1':
    img_path = input("Enter the full path of the image: ").strip()
    if not os.path.isfile(img_path):
        print("File does not exist:", img_path)
    else:
        pred_score, pred_label = preprocess_and_predict(img_path)
        if pred_score is not None:
            print(f"Prediction score: {pred_score}")
            print(f"Result: {pred_label}")

elif choice == '2':
    folder_path = input("Enter the full path of the image folder: ").strip()
    if not os.path.isdir(folder_path):
        print(" Folder does not exist:", folder_path)
    else:
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if not image_files:
            print(" No images found in folder:", folder_path)
        else:
            print(f" Found {len(image_files)} images.")
            results = []
            for filename in image_files:
                img_path = os.path.join(folder_path, filename)
                pred_score, pred_label = preprocess_and_predict(img_path)
                if pred_score is not None:
                    results.append({
                        'filename': filename,
                        'prediction_score': pred_score,
                        'label': pred_label
                    })
            if results:
                df = pd.DataFrame(results)
                output_csv = '/content/drive/MyDrive/cell_predictions.csv'
                df.to_csv(output_csv, index=False)
                print(f" Predictions saved to: {output_csv}")
            else:
                print(" No predictions made. Check for errors above.")
else:
    print(" Invalid choice. Please run again and choose 1 or 2.")


#9.Save the model

In [ ]:
model.save('/content/drive/MyDrive/malaria_inceptionv3_model.h5')
